# Determinando la mejor agrupación de las tiendas de Starbucks según su ubicación geográfica



## 1. Descripción del problema 

Se desea conocer cuál es la mejor forma de agrupar las tiendas de Starbucks, a nivel mundial, según su ubicación geográfica. Con esta información, se puede decidir cómo estructurar los cargos administrativos de la empresa; por ejemplo, algunas compañías toman Centroamérica como una región, mientras que otras tienen operaciones específicas para Costa Rica. 

Para esto, se utilizarán técnicas de aprendizaje no supervisado, específicamente de *clustering*, pues no se conoce con antelación cuál es el mejor agrupamiento de las tiendas. En este contexto, el *mejor agrupamiento* no necesariamente será el óptimo, sino que se refiere al *mejor agrupamiento obtenido por algún algoritmo* pues, como en diversos problemas de aprendizaje no supervisado, no se conoce cuál es la solución óptima al problema.

## 2. Preprocesamiento y limpieza de datos 

### 2.a Eliminar columnas redundantes o que no se utilizarán

### 2.b Cambiar tipo de datos en columnas de Longitud y Latitud

## 3. Transformación de datos

### 3.a Transformación de la columna Timezone 

Para que indique el continente de la tienda

### 3.b Creación de columna isUnitedStates a partir de Timezone

Para que indique si es de Estados Unidos o no

## 4. Análisis Exploratorio de Datos

### 4.a Estadística Descriptiva

### 4.b Particionamiento en clases

Se utiliza la columna isUnitedStates

## 5. Técnicas de segmentación

Explicar por qué sirven

### 5.a K-Means

### 5.b DBSCAN

## 6. Evaluación de modelos

### 6.a Silhouette Analysis - K-Means

### 6.b Silhouette Analysis - DBSCAN

## 7. Resultados

Gráfica + tabla

## 8. Análisis